<a href="https://colab.research.google.com/github/MuhammadIrzam447/MultiModel/blob/master/Valid_5(a).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !gdown https://drive.google.com/uc?id=1kl1NQvbcre7ktBmaxvdZ5fN_EmQQicBL # Joint

In [ ]:
# !gdown https://drive.google.com/uc?id=1F0-cWEZb32grArL7vBzqLwf_l_yF1EkT # Fused Dataset

In [ ]:
# !gdown https://drive.google.com/uc?id=1GH5VeZucpxLI5hX9c-Uwx5PAyBW-wuH3 # Model

In [ ]:
# !unzip /content/mulitmodal_joint_224x224.zip

In [ ]:
# !unzip /content/test.zip

# Load Model


In [1]:
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
import torchvision.models as models
from tqdm import tqdm

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
import os
save_dir = '/content/'
load_path = os.path.join(save_dir, 'fused_model.pth')

# Create an instance of the ResNet model
resnet = torchvision.models.resnet101(pretrained=False)
resnet.fc = nn.Linear(2048, 101) # Choose the number of output classses as per your model

# Load the saved model parameters
resnet.load_state_dict(torch.load(load_path))
# resnet.load_state_dict(torch.load(load_path, map_location=torch.device('cpu')))

resnet.eval()
resnet.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Partial Missing Modality Experiment

In [4]:
from datasets import load_dataset
from datasets import concatenate_datasets

In [5]:
# Load the Fused Dataset
ds_f = load_dataset("imagefolder", data_dir="/content/test", split="train")

Resolving data files:   0%|          | 0/22716 [00:00<?, ?it/s]

In [6]:
ds_f

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

In [7]:
# Desired Percentage = Total - Missing
import math
desired_percentage = 0.7

In [8]:
from collections import Counter

class_counts = Counter(ds_f['label'])
print(class_counts)

Counter({42: 238, 62: 238, 95: 237, 82: 236, 84: 236, 76: 235, 100: 235, 0: 234, 47: 234, 55: 234, 57: 234, 65: 234, 72: 234, 17: 233, 45: 233, 53: 233, 59: 233, 96: 233, 21: 232, 31: 232, 52: 232, 58: 232, 81: 232, 98: 232, 10: 231, 14: 231, 33: 231, 63: 231, 67: 231, 12: 230, 23: 230, 70: 230, 92: 230, 30: 229, 44: 229, 49: 229, 93: 229, 29: 228, 32: 228, 36: 228, 51: 228, 71: 228, 87: 228, 89: 228, 11: 227, 15: 227, 64: 227, 77: 227, 83: 227, 2: 226, 8: 226, 86: 226, 4: 225, 7: 225, 27: 225, 50: 225, 73: 225, 5: 224, 6: 224, 16: 224, 22: 224, 24: 224, 28: 224, 46: 224, 66: 224, 80: 224, 41: 223, 48: 223, 56: 223, 69: 223, 94: 223, 97: 223, 3: 222, 18: 222, 1: 221, 26: 221, 13: 220, 19: 220, 25: 220, 79: 220, 20: 219, 88: 219, 75: 218, 91: 218, 68: 217, 39: 216, 61: 215, 85: 215, 9: 214, 34: 213, 90: 213, 37: 212, 40: 212, 43: 212, 74: 211, 99: 211, 78: 210, 38: 208, 54: 203, 35: 202, 60: 197})


In [9]:
selected_indices = {label: [] for label in class_counts.keys()}
print(selected_indices)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: [], 36: [], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [], 46: [], 47: [], 48: [], 49: [], 50: [], 51: [], 52: [], 53: [], 54: [], 55: [], 56: [], 57: [], 58: [], 59: [], 60: [], 61: [], 62: [], 63: [], 64: [], 65: [], 66: [], 67: [], 68: [], 69: [], 70: [], 71: [], 72: [], 73: [], 74: [], 75: [], 76: [], 77: [], 78: [], 79: [], 80: [], 81: [], 82: [], 83: [], 84: [], 85: [], 86: [], 87: [], 88: [], 89: [], 90: [], 91: [], 92: [], 93: [], 94: [], 95: [], 96: [], 97: [], 98: [], 99: [], 100: []}


In [10]:
# Iterate through the dataset to select indices for each class
for i, label in enumerate(ds_f["label"]):
    if len(selected_indices[label]) < math.ceil(class_counts[label] * desired_percentage):
        selected_indices[label].append(i)

In [11]:
# selected_indices

In [12]:
# Flatten the selected indices list
selected_indices = [idx for indices in selected_indices.values() for idx in indices]

In [13]:
# selected_indices

In [14]:
# Filter the dataset to select the desired samples
fused_ds = ds_f.select(selected_indices)

In [15]:
fused_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 15946
})

In [16]:
# Load the Joint Dataset and Filter out Just the Images
ds_joint = load_dataset("imagefolder", data_dir="/content/mulitmodal/test", split="train")

Resolving data files:   0%|          | 0/45432 [00:00<?, ?it/s]

In [17]:
# Use _4.png for Filtering out Just Images

import os
def filter_funtion(example):
    img = example["image"]
    filename = os.path.basename(img.filename)

    return filename.endswith("_4.png")

In [18]:
ds_imgs = ds_joint.filter(filter_funtion)

In [19]:
ds_imgs

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

In [20]:
# for i in range(10,100):
#     sample = ds_imgs[i]
#     img = sample["image"]
#     filename = os.path.basename(img.filename)
#     print(f"File Name: {filename}")

In [21]:
all_indices = range(len(ds_imgs))
print(all_indices)

range(0, 22716)


In [22]:
remaining_indices = [idx for idx in all_indices if idx not in selected_indices]

In [23]:
# remaining_indices

In [24]:
missing_ds = ds_imgs.select(remaining_indices)

In [25]:
missing_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 6770
})

In [26]:
# Combine Both Datasets

In [27]:
ds = concatenate_datasets([fused_ds, missing_ds])

In [28]:
ds

Dataset({
    features: ['image', 'label'],
    num_rows: 22716
})

# Pre-Processing and Validation Loop

In [29]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [30]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  std=[0.229, 0.224, 0.225])])

In [31]:
class CustomDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]["image"]
        label = self.dataset[idx]["label"]

        if self.transform:
            image = self.transform(image)

        return {"image": image, "label": label}

In [32]:
dataset = CustomDataset(ds, transform=transform)

In [33]:
batch_size = 32
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [34]:
true_labels = []
predicted_labels = []

resnet.eval()

with torch.no_grad():
    for batch in test_loader:
        inputs = batch["image"]
        labels = batch["label"]

        inputs = inputs.to(device)

        outputs = resnet(inputs)

        _, predicted = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

In [35]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')
# AUROC_score = roc_auc_score(true_labels, predicted_labels)

In [36]:
print("Accuracy: ", accuracy)
print("precision: ", precision)
print("f1_score: ", f1)
# print("AUROC_score: ", AUROC_score)

Accuracy:  0.8143599225215707
precision:  0.8194546178897751
f1_score:  0.8147715369630628


In [37]:
print(classification_report(true_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.84      0.81      0.82       234
           1       0.78      0.86      0.82       221
           2       0.78      0.95      0.86       226
           3       0.80      0.81      0.80       222
           4       0.67      0.56      0.61       225
           5       0.87      0.89      0.88       224
           6       0.86      0.79      0.82       224
           7       0.93      0.77      0.84       225
           8       0.88      0.79      0.83       226
           9       0.81      0.82      0.82       214
          10       0.90      0.88      0.89       231
          11       0.83      0.94      0.88       227
          12       0.88      0.82      0.85       230
          13       0.88      0.87      0.88       220
          14       0.74      0.80      0.77       231
          15       0.88      0.80      0.84       227
          16       0.67      0.71      0.69       224
          17       0.86    

In [38]:
cm = confusion_matrix(true_labels, predicted_labels)
print(cm)

[[189   1   4 ...   0   0   2]
 [  0 191   1 ...   0   0   0]
 [  1   0 215 ...   1   0   1]
 ...
 [  2   0   1 ... 199   0   1]
 [  0   0   0 ...   0 139   0]
 [  1   1   0 ...   0   0 205]]
